# Aplicação do Algoritmo A* para planejamento de visita à pontos turísticos na Itália

## Introdução:

### Motivação:

Este projeto tem como objetivo apresentar as características do algoritmo A* (A-Estrela) e propor a sua utilização para a resolução de um problema simples.

### Seleção do problema e estados objetivos:

Domínio do problema: Identificar a melhor rota entre os três pontos turisticos da Itália mais bem avaliados pelo site TripAdvisor.

De acordo com o TripAdvisor (plataforma digital reconhecida como o maior site de viagens do mundo), os pontos turísticos mais bem avaliados são:
- Sassi di Matera (Província de Taranto)
- St Peter's Basílica (Roma)
- Gardaland Park (Verona)

<img src="https://guiati9.dev/img/posts_img/tripadvisor.png"/>

Sendo assim, a solução proposta deve apresentar o caminho menos custoso à ser percorrido para um turista que pretenda visitar estes três pontos.

Considerando o contexto de agentes de busca, é possível dizer que os três destinos podem ser considerados os estados objetivos do problema, desde que ao fim da execução do algoritmo, todos tenham sido visitados. Enquanto, para fins práticos o ponto de partida será considerado o local de destino que apresenta o menor preço de passagem aérea entre São Paulo (GRU) e Itália. No dia 10 de junho de 2020, o vôo mais barato encontrado na plataforma Google Flights foi com destino à Venice (Veneza), que passa à ser então o ponto de partida do algoritmo (estado inicial do problema).

<img src="https://guiati9.dev/img/posts_img/gflights.png"/>

## Metodologia:


### Estrutura do agente:

O agente de busca neste problema pode ser considerado o algoritmo de busca A* que é implementado pela função ```astar_search()```, definida como:

In [1]:
start_node_value = 0
def astar_search(graph, start, end, start_node_value, heuristics):
    # Create lists for open nodes and closed nodes
    open = []
    closed = []
    opened_list = []
    closed_list = []
    opened_list_entry = []
    closed_list_entry = []
    next_start_node_value = 0
    nodes_values = []

    # Create a start node and an goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)

    # Add the start node
    open.append(start_node)
    #To save last execution g value
    start_node.g = start_node.g + start_node_value
    # Loop until the open list is empty
    while len(open) > 0:
        opened_list_entry = []
        closed_list_entry = []

        # Sort the open list to get the node with the lowest cost first
        open.sort()

        # Get the node with the lowest cost
        current_node = open.pop(0)

        # Add the current node to the closed list
        closed.append(current_node)
        closed_list_entry.append(current_node.name)
        closed_list.append(closed_list_entry)
        
        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name + ': ' + str(current_node.g))
                nodes_values.append(current_node.g)
                current_node = current_node.parent
            path.append(start_node.name + ': ' + str(start_node.g))
            # Return reversed path
            next_start_node_value = nodes_values[0]
            return path[::-1], opened_list, closed_list, next_start_node_value

        # Get neighbours
        neighbors = graph.get(current_node.name)
        
        # Loop neighbors
        for key, value in neighbors.items():

            # Create a neighbor node
            neighbor = Node(key, current_node)

            # Check if the neighbor is in the closed list
            if(neighbor in closed):
                continue

            # Calculate full path cost

            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            # Check if neighbor is in open list and if it has a lower f value
            if(add_to_open(open, neighbor) == True):
                # Everything is green, add neighbor to open list
                open.append(neighbor)
                opened_list_entry.append(neighbor.name)
        opened_list.append(opened_list_entry)

    # Return None, no path is found
    return None

In [2]:
# Check if a neighbor should be added to open list
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

def print_results(path,opened_list, closed_list):
    print('Caminho (Nó:Custo acumulado):')
    print(path)
    print("\n")
    #Printing A* executions/open/closed nodes
    opened_list.pop()
    print('Lista de novos nós abertos:')
    print(opened_list)
    print("\n")
    print('Lista de novos nós fechados:')
    print(closed_list)
    return

def gather_paths(path, new_path_piece):
  for i in new_path_piece:
    path.append(i)
  return path

### Estrutura de dados:

BREVE EXPLICAÇÃO DA ESTRUTURA DE DADOS

In [3]:
# This class represent a graph
class Graph:

    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist

    # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance

    # Get neighbors or a neighbor
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

In [4]:
# This class represent a node
class Node:

    # Initialize the class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost

    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name

    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f

### Função heurística adotada:

#### Distâncias em linha reta/100 + Preço de Hotel/10

In [5]:
import pandas as pd
import numpy as np
dist_linha_reta = pd.read_csv("distancias_estacoes_italia.csv", index_col = 0)
precos_hotel = pd.read_csv("precos_hotel.csv",)


# dividing all by 100 to reduce heuristics impact (if not it would become inadmissible)
precos_hotel = precos_hotel
tabela_heuristicas = dist_linha_reta

precos_hotel

,Unnamed: 0,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,...,Rome,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,ReggiodiCalabria,Unnamed: 21
0,NaN,326,75,84,155,170,145,178,178,117,...,109,238,95,223,106,237,167,145,150,NaN


In [6]:
tabela_heuristicas['Bolzano'] = tabela_heuristicas['Bolzano'].apply(lambda x: x*precos_hotel['Bolzano']/2000)
tabela_heuristicas['Verona'] = tabela_heuristicas['Verona'].apply(lambda x: x*precos_hotel['Verona']/2000)
tabela_heuristicas['Milano'] = tabela_heuristicas['Milano'].apply(lambda x: x*precos_hotel['Milano']/2000)
tabela_heuristicas['Torino'] = tabela_heuristicas['Torino'].apply(lambda x: x*precos_hotel['Torino']/2000)
tabela_heuristicas['Trieste'] = tabela_heuristicas['Trieste'].apply(lambda x: x*precos_hotel['Trieste']/2000)
tabela_heuristicas['Venice'] = tabela_heuristicas['Venice'].apply(lambda x: x*precos_hotel['Venice']/2000)
tabela_heuristicas['Bologna'] = tabela_heuristicas['Bologna'].apply(lambda x: x*precos_hotel['Bologna']/2000)
tabela_heuristicas['Genoa'] = tabela_heuristicas['Genoa'].apply(lambda x: x*precos_hotel['Genoa']/2000)
tabela_heuristicas['Pisa'] = tabela_heuristicas['Pisa'].apply(lambda x: x*precos_hotel['Pisa']/2000)
tabela_heuristicas['Florence'] = tabela_heuristicas['Florence'].apply(lambda x: x*precos_hotel['Florence']/2000)
tabela_heuristicas['Ancona'] = tabela_heuristicas['Ancona'].apply(lambda x: x*precos_hotel['Ancona']/2000)
tabela_heuristicas['Rome'] = tabela_heuristicas['Rome'].apply(lambda x: x*precos_hotel['Rome']/2000)
tabela_heuristicas['Pescara'] = tabela_heuristicas['Pescara'].apply(lambda x: x*precos_hotel['Pescara']/2000)
tabela_heuristicas['Naples'] = tabela_heuristicas['Naples'].apply(lambda x: x*precos_hotel['Naples']/2000)
tabela_heuristicas['Foggia'] = tabela_heuristicas['Foggia'].apply(lambda x: x*precos_hotel['Foggia']/2000)
tabela_heuristicas['Bari'] = tabela_heuristicas['Bari'].apply(lambda x: x*precos_hotel['Bari']/2000)
tabela_heuristicas['Brindisi'] = tabela_heuristicas['Brindisi'].apply(lambda x: x*precos_hotel['Brindisi']/2000)
tabela_heuristicas['Lecce'] = tabela_heuristicas['Lecce'].apply(lambda x: x*precos_hotel['Lecce']/2000)
tabela_heuristicas['Taranto'] = tabela_heuristicas['Taranto'].apply(lambda x: x*precos_hotel['Taranto']/2000)
tabela_heuristicas['Reggio di Calabria'] = tabela_heuristicas['Reggio di Calabria'].apply(lambda x: x*precos_hotel['ReggiodiCalabria']/2000)


tabela_heuristicas = tabela_heuristicas.round(2)
tabela_heuristicas

,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,Florence,Ancona,Rome,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,Reggio di Calabria
Bolzano,0.00,4.65,8.78,25.19,17.60,10.08,19.85,26.43,18.43,20.43,35.20,28.23,59.62,31.73,72.59,39.43,98.59,73.31,59.52,74.62
Verona,20.21,0.00,5.80,20.38,18.53,7.10,9.52,17.71,11.52,12.40,27.79,22.40,50.10,27.36,64.11,35.83,92.19,67.89,54.23,67.58
Milano,34.07,5.18,0.00,10.15,30.26,17.62,17.80,10.68,12.75,16.68,38.90,26.16,62.36,31.30,75.49,41.66,105.70,77.24,62.13,73.12
Torino,52.98,9.86,5.50,0.00,40.80,26.46,26.34,11.12,15.44,21.31,47.78,28.61,71.40,33.87,83.96,45.84,115.06,83.83,67.50,76.65
Trieste,33.74,8.18,14.95,37.20,0.00,8.26,20.29,35.96,20.01,19.30,22.13,23.38,42.60,25.41,54.30,29.84,77.38,57.62,46.40,63.98
Venice,22.66,3.68,10.21,28.29,9.69,0.00,11.57,25.81,14.39,13.67,21.84,21.42,43.55,25.36,57.20,32.06,83.07,61.71,49.30,64.58
Bologna,36.35,4.01,8.40,22.94,19.38,9.43,0.00,17.09,6.79,5.63,19.40,16.57,38.67,22.37,53.52,31.11,81.76,60.54,47.63,59.78
Genoa,48.41,7.46,5.04,9.69,34.34,21.02,17.09,0.00,8.25,13.33,36.76,21.96,57.12,28.12,70.36,39.43,100.61,73.81,58.72,67.50
Pisa,51.34,7.39,9.16,20.46,29.07,17.84,10.32,12.55,0.00,4.62,24.28,14.39,40.46,21.38,54.52,32.01,84.02,61.96,48.43,57.30
Florence,49.72,6.94,10.46,24.64,24.48,14.79,7.48,17.71,4.04,0.00,17.65,12.64,33.44,19.43,48.50,28.94,77.26,57.28,44.44,54.68


Além da obtenção e organização dos dados referentes às distâncias em linha reta de uma estação de trem à outra, a quantidade de caminhos possíveis exige que a função referente ao agente de busca seja executada diversas vezes. Para facilitar a manipulação do agente de busca, foi implementada a seguinte função, que tem como objetivo buscar, de forma automática, na tabela de distâncias em linha reta o valor das funções heurísticas de acordo com o destino escolhido.

In [7]:
stations_list = list(tabela_heuristicas.columns.values)
heuristics_dict = tabela_heuristicas.to_dict()
heuristics = {}

def set_heuristics(station_name):
    station_index = stations_list.index(station_name)
    heuristics = [value for value in heuristics_dict.values()][station_index]
    return heuristics

#testing if heuristics are being correctly shown
heuristics = set_heuristics('Verona')
print(heuristics)

{'Bolzano': 4.65, 'Verona': 0.0, 'Milano': 5.18, 'Torino': 9.86, 'Trieste': 8.18, 'Venice': 3.68, 'Bologna': 4.01, 'Genoa': 7.46, 'Pisa': 7.39, 'Florence': 6.94, 'Ancona': 10.69, 'Rome': 15.41, 'Pescara': 15.79, 'Naples': 21.6, 'Foggia': 21.56, 'Bari': 25.35, 'Brindisi': 29.18, 'Lecce': 30.49, 'Taranto': 28.05, 'Reggio di Calabria': 33.79}


#### Disposição das estações e o custo aproximado das rotas de trem na Itália

Custo de rotas           |  Principais estações
:-------------------------:|:-------------------------:
<img src="https://guiati9.dev/img/posts_img/cost-map-italy.png" width="400px"/>  |  <img src="https://guiati9.dev/img/posts_img/italy-train-map.png" width="400px"/>



#### Grafo gerado a partir do custo aproximado e mapa de estações

<img src="https://guiati9.dev/img/posts_img/italy_routes_prices.png" width="600px"/>

INSERIR IMAGEM DO GRAFO CRIADO

In [8]:
# Create a graph
graph = Graph()

graph.connect('Reggio di Calabria', 'Taranto', 149)
graph.connect('Reggio di Calabria', 'Naples', 323)
graph.connect('Lecce', 'Brindisi', 25)
graph.connect('Brindisi', 'Bari', 99)
graph.connect('Bari', 'Taranto', 49)
graph.connect('Bari', 'Foggia', 124)
graph.connect('Taranto', 'Naples', 149)
graph.connect('Naples', 'Foggia', 248)
graph.connect('Foggia', 'Pescara', 124)
graph.connect('Naples', 'Rome', 248)
graph.connect('Rome', 'Pescara', 74)
graph.connect('Pescara', 'Ancona', 74)
graph.connect('Rome', 'Ancona', 124)
graph.connect('Rome', 'Florence', 248)
graph.connect('Rome', 'Pisa', 248)
graph.connect('Pisa', 'Florence', 49)
graph.connect('Florence', 'Bologna', 149)
graph.connect('Bologna', 'Ancona', 174)
graph.connect('Pisa', 'Genoa', 149)
graph.connect('Genoa', 'Bologna', 124)
graph.connect('Genoa', 'Torino', 124)
graph.connect('Genoa', 'Milano', 124)
graph.connect('Milano', 'Bologna', 223)
graph.connect('Bologna', 'Verona', 99)
graph.connect('Bologna', 'Venice', 174)
graph.connect('Venice', 'Trieste', 74)
graph.connect('Venice', 'Verona', 124)
graph.connect('Verona', 'Milano', 124)
graph.connect('Milano', 'Torino', 174)
graph.connect('Verona', 'Bolzano', 124)

# Create graph connections (Actual distance)


# Make graph undirected, create symmetric connections
graph.make_undirected()

## 3. Resultados

Considerando o estado inicial do problema e os possíveis estados finais, constata-se que o melhor caminho está contido entre as seguintes possibilidades:

<img src="https://guiati9.dev/img/posts_img/Untitled%20Diagram.png" style="margin-left:auto; margin-right:auto;"/>


- **Hipótese 1:** Venice -> ... -> Rome -> ... -> Taranto -> ... -> Verona
- **Hipótese 2:** Venice -> ... -> Rome -> ... -> Verona -> ... -> Taranto
- **Hipótese 3:** Venice -> ... -> Taranto -> ... -> Verona -> ... -> Rome
- **Hipótese 4:** Venice -> ... -> Taranto -> ... -> Rome -> ... -> Verona
- **Hipótese 5:** Venice -> ... -> Verona -> ... -> Taranto -> ... -> Rome
- **Hipótese 6:** Venice -> ... -> Verona -> ... -> Rome -> ... -> Taranto
---

### Hipótese 1:

*Venice -> ... -> Rome -> ... -> Taranto -> ... -> Verona*

In [9]:
rome_heuristics = set_heuristics('Rome')
taranto_heuristics = set_heuristics('Taranto')
verona_heuristics = set_heuristics('Verona')
#Venice to Rome
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Rome', start_node_value, rome_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value, taranto_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Verona', start_node_value, verona_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)

Caminho
['Venice: 0', 'Bologna: 174', 'Ancona: 348', 'Rome: 472', 'Pescara: 546', 'Foggia: 670', 'Bari: 794', 'Taranto: 843', 'Bari: 892', 'Foggia: 1016', 'Pescara: 1140', 'Ancona: 1214', 'Bologna: 1388', 'Verona: 1487']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], [], ['Milano', 'Bolzano'], ['Ancona', 'Florence', 'Genoa'], ['Torino'], [], ['Torino', 'Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], ['Foggia'], [], [], []], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'Naples'], ['Foggia'], ['Bologna'], ['Bari'], ['Reggio di Calabria', 'Taranto'], [], ['Genoa'], ['Taranto', 'Brindisi'], ['Verona', 'Venice', 'Milano']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Brindisi'], ['Rome'], ['Lecce'], [], ['Pescara'], [], ['Ancona', 'Rome'], ['Bologna'], ['Florence', 'Pisa'], [], ['Verona', 'Venice', 'Genoa', 'Milano'], [], []]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Verona'], ['Bologna'], ['Milano'], ['Bolzano'], ['Genoa'], ['Florence'], ['Ancona'], ['Pisa'], ['P

### Hipótese 2:

*Venice -> ... -> Rome -> ... -> Verona -> ... -> Taranto*

In [10]:
#Venice to Rome
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Rome', start_node_value,rome_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Verona', start_node_value, verona_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Taranto', start_node_value, taranto_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 174', 'Ancona: 348', 'Rome: 472', 'Ancona: 596', 'Bologna: 770', 'Verona: 869', 'Bologna: 968', 'Ancona: 1142', 'Pescara: 1216', 'Foggia: 1340', 'Bari: 1464', 'Taranto: 1513']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], [], ['Milano', 'Bolzano'], ['Ancona', 'Florence', 'Genoa'], ['Torino'], [], ['Torino', 'Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], ['Foggia'], [], [], []], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'Naples'], ['Foggia'], ['Bologna'], ['Bari'], [], ['Genoa'], ['Reggio di Calabria', 'Taranto'], ['Verona', 'Venice', 'Milano'], ['Taranto', 'Brindisi']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'Florence', 'Genoa'], ['Trieste'], [], ['Torino'], [], ['Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], [], ['Foggia'], [], ['Naples'], ['Bari'], [], ['Taranto', 'Brindisi']]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Verona'], ['Bologna'], ['Milano'], ['Bolzano'], ['Genoa'], ['Florence'], ['Ancona'], ['Pisa']

### Hipótese 3:

*Venice -> ... -> Taranto -> ... -> Verona -> ... -> Rome*

In [11]:
#Venice to Rome
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Taranto', start_node_value,taranto_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Verona', start_node_value, verona_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value, rome_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 174', 'Ancona: 348', 'Pescara: 422', 'Foggia: 546', 'Bari: 670', 'Taranto: 719', 'Bari: 768', 'Foggia: 892', 'Pescara: 1016', 'Ancona: 1090', 'Bologna: 1264', 'Verona: 1363', 'Bologna: 1462', 'Ancona: 1636', 'Rome: 1760']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], [], ['Milano', 'Bolzano'], ['Ancona', 'Florence', 'Genoa'], [], ['Torino'], ['Torino', 'Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], ['Foggia'], [], [], [], ['Naples'], ['Bari'], [], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Brindisi'], ['Rome'], ['Lecce'], [], ['Pescara'], [], ['Ancona', 'Rome'], ['Bologna'], ['Florence', 'Pisa'], [], ['Verona', 'Venice', 'Genoa', 'Milano'], [], []], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'Florence', 'Genoa'], ['Trieste'], ['Torino'], [], [], ['Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], [], ['Foggia'], []]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Verona'], ['Bologna'], ['B

### Hipótese 4:

*Venice -> ... -> Taranto -> ... -> Rome -> ... -> Verona*

In [12]:
#Venice to Taranto
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Taranto', start_node_value, taranto_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Rome', start_node_value, rome_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Verona', start_node_value, verona_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Bologna: 174', 'Ancona: 348', 'Pescara: 422', 'Foggia: 546', 'Bari: 670', 'Taranto: 719', 'Bari: 768', 'Foggia: 892', 'Pescara: 1016', 'Rome: 1090', 'Ancona: 1214', 'Bologna: 1388', 'Verona: 1487']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], [], ['Milano', 'Bolzano'], ['Ancona', 'Florence', 'Genoa'], [], ['Torino'], ['Torino', 'Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], ['Foggia'], [], [], [], ['Naples'], ['Bari'], [], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Brindisi'], ['Rome'], ['Lecce'], [], ['Pescara'], [], ['Ancona', 'Rome']], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'Naples'], ['Foggia'], ['Bologna'], ['Bari'], [], ['Genoa'], ['Reggio di Calabria', 'Taranto'], ['Verona', 'Venice', 'Milano'], ['Taranto', 'Brindisi']]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Verona'], ['Bologna'], ['Bolzano'], ['Milano'], ['Genoa'], ['Florence'], ['Ancona'], ['Pisa'], ['Pescara'], ['Torino'], ['Torino'], ['P

### Hipótese 5:

*Venice -> ... -> Verona -> ... -> Taranto -> ... -> Rome*

In [13]:
#Venice to Taranto
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Verona', start_node_value, verona_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Taranto', start_node_value, taranto_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Taranto', 'Rome', start_node_value, rome_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Verona: 124', 'Bologna: 223', 'Ancona: 397', 'Pescara: 471', 'Foggia: 595', 'Bari: 719', 'Taranto: 768', 'Bari: 817', 'Foggia: 941', 'Pescara: 1065', 'Rome: 1139']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], []], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'Florence', 'Genoa'], ['Trieste'], [], ['Torino'], [], ['Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], [], ['Foggia'], [], ['Naples'], ['Bari'], [], ['Taranto', 'Brindisi']], [['Naples', 'Reggio di Calabria', 'Bari'], ['Foggia', 'Brindisi'], ['Rome'], ['Lecce'], [], ['Pescara'], [], ['Ancona', 'Rome']]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Verona']], [['Verona'], ['Bologna'], ['Venice'], ['Bolzano'], ['Milano'], ['Trieste'], ['Genoa'], ['Florence'], ['Ancona'], ['Pisa'], ['Torino'], ['Pescara'], ['Pisa'], ['Rome'], ['Foggia'], ['Rome'], ['Bari'], ['Taranto']], [['Taranto'], ['Bari'], ['Naples'], ['Brindisi'], ['Reggio di Calabria'], ['Foggia'], ['Lecce'], ['Pescara'], ['Rom

### Hipótese 6:

*Venice -> ... -> Verona -> ... -> Rome -> ... -> Taranto*

In [14]:
#Venice to Taranto
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Verona', start_node_value, verona_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value, rome_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value, taranto_heuristics)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)



Caminho
['Venice: 0', 'Verona: 124', 'Bologna: 223', 'Ancona: 397', 'Rome: 521', 'Pescara: 595', 'Foggia: 719', 'Bari: 843', 'Taranto: 892']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], []], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'Florence', 'Genoa'], ['Trieste'], ['Torino'], [], [], ['Pisa'], ['Rome', 'Pisa'], ['Rome', 'Pescara'], [], [], ['Foggia'], []], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'Naples'], ['Foggia'], ['Bologna'], ['Bari'], ['Reggio di Calabria', 'Taranto'], [], ['Genoa'], ['Taranto', 'Brindisi'], ['Verona', 'Venice', 'Milano']]]
Lista de fechados:
[[['Venice'], ['Trieste'], ['Verona']], [['Verona'], ['Bologna'], ['Venice'], ['Milano'], ['Bolzano'], ['Trieste'], ['Genoa'], ['Florence'], ['Ancona'], ['Pisa'], ['Torino'], ['Pescara'], ['Pisa'], ['Rome']], [['Rome'], ['Pescara'], ['Ancona'], ['Foggia'], ['Naples'], ['Florence'], ['Pisa'], ['Bari'], ['Bologna'], ['Taranto']]]
Custo total:
892


### Análise dos resultados

De acordo com os resultados obtidos nas execuções do algoritmo A*, a hipótese que apresentou o melhor caminho é a Hipótese 6

#### Caminho: Venice -> Verona -> Bologna -> Ancona -> Rome -> Pescara -> Foggia -> Bari - > Taranto
#### Custo: 892

## Admissibilidade da Heurística

### Comparação com heurística não admissível

In [15]:
dist_linha_reta = pd.read_csv("distancias_estacoes_italia.csv", index_col = 0)
tabela_heuristicas = dist_linha_reta


tabela_heuristicas['Milano'] = tabela_heuristicas['Milano'].apply(lambda x: x*precos_hotel['Milano'])
tabela_heuristicas['Bologna'] = tabela_heuristicas['Bologna'].apply(lambda x: x*precos_hotel['Florence'])
tabela_heuristicas.round(2)
heuristics_dict = {}
heuristics_dict = tabela_heuristicas.to_dict()
tabela_heuristicas = tabela_heuristicas.T
tabela_heuristicas['Bologna'] = tabela_heuristicas['Bologna'].apply(lambda x: x*precos_hotel['Florence'])
tabela_heuristicas


,Bolzano,Verona,Milano,Torino,Trieste,Venice,Bologna,Genoa,Pisa,Florence,Ancona,Rome,Pescara,Naples,Foggia,Bari,Brindisi,Lecce,Taranto,Reggio di Calabria
Bolzano,0,124,209,325,207,139,29882,297,315,305,361,518,501,668,651,744,832,878,821,995
Verona,124,0,138,263,218,98,14338,199,197,185,285,411,421,576,575,676,778,813,748,901
Milano,17556,11592,0,11004,29904,20412,2251200,10080,18312,20916,33516,40320,44016,55356,56868,66024,74928,77700,71988,81900
Torino,325,263,131,0,480,365,39664,125,264,318,490,525,600,713,753,865,971,1004,931,1022
Trieste,207,218,356,480,0,114,30552,404,342,288,227,429,358,535,487,563,653,690,640,853
Venice,139,98,243,365,114,0,17420,290,246,204,224,393,366,534,513,605,701,739,680,861
Bologna,29882,14338,26800,39664,30552,17420,0,25728,15544,11256,26666,40736,43550,63114,64320,78658,92460,97150,88038,106798
Genoa,297,199,120,125,404,290,25728,0,141,199,377,403,480,592,631,744,849,884,810,900
Pisa,315,197,218,264,342,246,15544,141,0,69,249,264,340,450,489,604,709,742,668,764
Florence,305,185,249,318,288,204,11256,199,69,0,181,232,281,409,435,546,652,686,613,729


In [16]:

teste = set_heuristics('Bologna')
print(teste)
for i in teste:
    i = 0

print(teste)

{'Bolzano': 29882, 'Verona': 14338, 'Milano': 26800, 'Torino': 39664, 'Trieste': 30552, 'Venice': 17420, 'Bologna': 0, 'Genoa': 25728, 'Pisa': 15544, 'Florence': 11256, 'Ancona': 26666, 'Rome': 40736, 'Pescara': 43550, 'Naples': 63114, 'Foggia': 64320, 'Bari': 78658, 'Brindisi': 92460, 'Lecce': 97150, 'Taranto': 88038, 'Reggio di Calabria': 106798}
{'Bolzano': 29882, 'Verona': 14338, 'Milano': 26800, 'Torino': 39664, 'Trieste': 30552, 'Venice': 17420, 'Bologna': 0, 'Genoa': 25728, 'Pisa': 15544, 'Florence': 11256, 'Ancona': 26666, 'Rome': 40736, 'Pescara': 43550, 'Naples': 63114, 'Foggia': 64320, 'Bari': 78658, 'Brindisi': 92460, 'Lecce': 97150, 'Taranto': 88038, 'Reggio di Calabria': 106798}


In [19]:




#Venice to Taranto
new_path_piece = []
open_list = []
close_list = []
start_node_value = 0
path = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Venice', 'Verona', start_node_value, teste)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
path.pop()

#Rome to Taranto
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Verona', 'Rome', start_node_value, teste)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)
new_path_piece = []
path.pop()

#Taranto to Verona
new_path_piece = []
new_path_piece, opened_list, closed_list, start_node_value = astar_search(graph, 'Rome', 'Taranto', start_node_value, teste)
gather_paths(path, new_path_piece)
open_list.append(opened_list)
close_list.append(closed_list)

#results
print('Caminho')
print(path)
print('Lista de abertos:')
print(open_list)
print('Lista de fechados:')
print(close_list)
print('Custo total:')
print(start_node_value)


Caminho
['Venice: 0', 'Verona: 124', 'Bologna: 223', 'Ancona: 397', 'Rome: 521', 'Pescara: 595', 'Foggia: 719', 'Bari: 843', 'Taranto: 892']
Lista de abertos:
[[['Trieste', 'Verona', 'Bologna'], ['Ancona', 'Florence', 'Genoa', 'Milano'], ['Rome', 'Pisa']], [['Milano', 'Bolzano', 'Bologna', 'Venice'], ['Ancona', 'Florence', 'Genoa'], ['Rome', 'Pisa'], [], ['Trieste'], ['Torino'], ['Torino'], ['Rome', 'Pescara'], [], [], [], []], [['Pescara', 'Ancona', 'Florence', 'Pisa', 'Naples'], ['Bologna'], ['Verona', 'Venice', 'Genoa', 'Milano'], ['Milano', 'Bolzano'], ['Genoa'], ['Trieste'], ['Torino', 'Milano'], [], [], [], [], [], [], [], [], ['Foggia'], ['Reggio di Calabria', 'Taranto'], ['Bari'], ['Taranto', 'Brindisi']]]
Lista de fechados:
[[['Venice'], ['Bologna'], ['Florence'], ['Verona']], [['Verona'], ['Bologna'], ['Florence'], ['Pisa'], ['Venice'], ['Genoa'], ['Milano'], ['Ancona'], ['Bolzano'], ['Trieste'], ['Torino'], ['Torino'], ['Rome']], [['Rome'], ['Florence'], ['Bologna'], ['Veron

## Conclusão


## Referências
